<a href="https://colab.research.google.com/github/BlackCurrantDS/DBSE_Project/blob/main/FP_Growth_StepbyStep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [80]:
from collections import defaultdict
from itertools import chain, combinations

In [81]:
itemSetList = [['eggs', 'bacon', 'soup'],
                ['eggs', 'bacon', 'apple'],
                ['soup', 'bacon', 'banana']]

In [82]:
minsup=0.5
minConf=0.5

In [83]:
def getFrequencyFromList(itemSetList):
    frequency = [1 for i in range(len(itemSetList))]
    return frequency

In [84]:
frequency = getFrequencyFromList(itemSetList)
frequency

[1, 1, 1]

In [85]:
class Node:
    def __init__(self, itemName, frequency, parentNode):
        self.itemName = itemName
        self.count = frequency
        self.parent = parentNode
        self.children = {}
        self.next = None

    def increment(self, frequency):
        self.count += frequency

    def display(self, ind=1):
        print('  ' * ind, self.itemName, ' ', self.count)
        for child in list(self.children.values()):
            child.display(ind+1)

In [86]:
def updateTree(item, treeNode, headerTable, frequency):
    if item in treeNode.children:
        # If the item already exists, increment the count
        treeNode.children[item].increment(frequency)
    else:
        # Create a new branch
        newItemNode = Node(item, frequency, treeNode)
        treeNode.children[item] = newItemNode
        # Link the new branch to header table
        updateHeaderTable(item, newItemNode, headerTable)

    return treeNode.children[item]

In [87]:
def updateHeaderTable(item, targetNode, headerTable):
    if(headerTable[item][1] == None):
        headerTable[item][1] = targetNode
    else:
        currentNode = headerTable[item][1]
        # Traverse to the last node then link it to the target
        while currentNode.next != None:
            currentNode = currentNode.next
        currentNode.next = targetNode

In [88]:
def ascendFPtree(node, prefixPath):
    if node.parent != None:
        prefixPath.append(node.itemName)
        ascendFPtree(node.parent, prefixPath)

In [89]:
def constructTree(itemSetList, frequency, minSup):
    headerTable = defaultdict(int)
    print(headerTable)
    # Counting frequency and create header table
    for idx, itemSet in enumerate(itemSetList):
        for item in itemSet:
            headerTable[item] += frequency[idx]
    print("Adding items to dictionary",headerTable)
    # Deleting items below minSup
    headerTable = dict((item, sup) for item, sup in headerTable.items() if sup >= minSup)
    if(len(headerTable) == 0):
        return None, None
    print("Remving items from dictionary below min sup",headerTable)
    # HeaderTable column [Item: [frequency, headNode]]
    for item in headerTable:
        headerTable[item] = [headerTable[item], None]
    print("After Formatting",headerTable)
    # Init Null head node, class Node 
    fpTree = Node('Null', 1, None) 
    # Update FP tree for each cleaned and sorted itemSet
    for idx, itemSet in enumerate(itemSetList):
        itemSet = [item for item in itemSet if item in headerTable]
        itemSet.sort(key=lambda item: headerTable[item][0], reverse=True)
        # Traverse from root to leaf, update tree with given item
        currentNode = fpTree
        for item in itemSet:
            currentNode = updateTree(item, currentNode, headerTable, frequency[idx])
    print("headerTable",headerTable)
    print("fpTree",fpTree)
    return fpTree, headerTable

In [90]:
def findPrefixPath(basePat, headerTable):
    # First node in linked list
    treeNode = headerTable[basePat][1] 
    condPats = []
    frequency = []
    while treeNode != None:
        prefixPath = []
        # From leaf node all the way to root
        ascendFPtree(treeNode, prefixPath)  
        if len(prefixPath) > 1:
            # Storing the prefix path and it's corresponding count
            condPats.append(prefixPath[1:])
            frequency.append(treeNode.count)

        # Go to next node
        treeNode = treeNode.next  
    return condPats, frequency

In [91]:
def mineTree(headerTable, minSup, preFix, freqItemList):
    # Sort the items with frequency and create a list
    sortedItemList = [item[0] for item in sorted(list(headerTable.items()), key=lambda p:p[1][0])]
    print("sortedItemList", sortedItemList) 
    # Start with the lowest frequency
    for item in sortedItemList:  
        # Pattern growth is achieved by the concatenation of suffix pattern with frequent patterns generated from conditional FP-tree
        newFreqSet = preFix.copy()
        print("newFreqSet",newFreqSet)
        newFreqSet.add(item)
        print("newFreqSet",newFreqSet)
        freqItemList.append(newFreqSet)
        print("freqItemList",freqItemList)
        # Find all prefix path, constrcut conditional pattern base
        conditionalPattBase, frequency = findPrefixPath(item, headerTable) 
        # Construct conditonal FP Tree with conditional pattern base
        conditionalTree, newHeaderTable = constructTree(conditionalPattBase, frequency, minSup) 
        if newHeaderTable != None:
            # Mining recursively on the tree
            mineTree(newHeaderTable, minSup,
                       newFreqSet, freqItemList)

In [92]:
def powerset(s):
    return chain.from_iterable(combinations(s, r) for r in range(1, len(s)))

def getSupport(testSet, itemSetList):
    count = 0
    for itemSet in itemSetList:
        if(set(testSet).issubset(itemSet)):
            count += 1
    return count

In [93]:
def associationRule(freqItemSet, itemSetList, minConf):
    rules = []
    for itemSet in freqItemSet:
        subsets = powerset(itemSet)
        print("subsets",subsets)
        itemSetSup = getSupport(itemSet, itemSetList)
        print("itemSetSup",itemSetSup)
        for s in subsets:
            confidence = float(itemSetSup / getSupport(s, itemSetList))
            print("confidence",confidence)
            if(confidence > minConf):
              rules.append([set(s), set(itemSet.difference(s)), confidence])
    print("rules", rules)
    return rules

In [94]:
def fpgrowth(itemSetList, minSupRatio, minConf):
    frequency = getFrequencyFromList(itemSetList)
    print(frequency)
    minSup = len(itemSetList) * minSupRatio
    print(minSup)
    fpTree, headerTable = constructTree(itemSetList, frequency, minSup)
    print("headerTable final",headerTable)
    print("fpTree final",fpTree)
    if(fpTree == None):
        print('No frequent item set')
    else:
        freqItems = []
        mineTree(headerTable, minSup, set(), freqItems)
        rules = associationRule(freqItems, itemSetList, minConf)
        return freqItems, rules

In [95]:
fpgrowth(itemSetList, minSupRatio=0.5, minConf=0.5)

[1, 1, 1]
1.5
defaultdict(<class 'int'>, {})
Adding items to dictionary defaultdict(<class 'int'>, {'eggs': 2, 'bacon': 3, 'soup': 2, 'apple': 1, 'banana': 1})
Remving items from dictionary below min sup {'eggs': 2, 'bacon': 3, 'soup': 2}
After Formatting {'eggs': [2, None], 'bacon': [3, None], 'soup': [2, None]}
headerTable {'eggs': [2, <__main__.Node object at 0x7fc633e63d30>], 'bacon': [3, <__main__.Node object at 0x7fc633e63390>], 'soup': [2, <__main__.Node object at 0x7fc633e63ac8>]}
fpTree <__main__.Node object at 0x7fc633e63240>
headerTable final {'eggs': [2, <__main__.Node object at 0x7fc633e63d30>], 'bacon': [3, <__main__.Node object at 0x7fc633e63390>], 'soup': [2, <__main__.Node object at 0x7fc633e63ac8>]}
fpTree final <__main__.Node object at 0x7fc633e63240>
sortedItemList ['eggs', 'soup', 'bacon']
newFreqSet set()
newFreqSet {'eggs'}
freqItemList [{'eggs'}]
defaultdict(<class 'int'>, {})
Adding items to dictionary defaultdict(<class 'int'>, {'bacon': 2})
Remving items from

([{'eggs'}, {'bacon', 'eggs'}, {'soup'}, {'bacon', 'soup'}, {'bacon'}],
 [[{'bacon'}, {'eggs'}, 0.6666666666666666],
  [{'eggs'}, {'bacon'}, 1.0],
  [{'soup'}, {'bacon'}, 1.0],
  [{'bacon'}, {'soup'}, 0.6666666666666666]])